## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, GRU, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import Callback, EarlyStopping

In [3]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

2022-11-30 14:27:24.592373: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-30 14:27:24.604254: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-30 14:27:24.604409: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


## Data Loading

In [4]:
train_df = pd.read_csv('../data/processed/train_data_processed.csv', index_col='id')
train_df.head()

,title,genre,description,year,language,processed_description_string
id,,,,,,
1,Oscar et la dame rose,drama,Listening in to a conversation between his do...,2009,English,listen convers doctor parent 10-year-old oscar...
2,Cupid,thriller,A brother and sister with a past incestuous r...,1997,English,brother sister past incestu relationship curre...
3,"Young, Wild and Wonderful",adult,As the bus empties the students for their fie...,1980,English,bu empti student field trip museum natur histo...
4,The Secret Sin,drama,To help their unemployed father make ends mee...,1915,English,help unemploy father make end meet edith twin ...
5,The Unrecovered,drama,The film's title refers not only to the un-re...,2007,English,film titl refer un-recov bodi ground zero also...


In [5]:
# Filter out non-English languages
print(train_df.shape)
train_df = train_df[train_df['language'] == 'English'].reset_index(drop=True)
print(train_df.shape)

(48790, 6)
(48790, 6)


In [6]:
unique_genres = train_df['genre'].unique()
genre_encoding = dict([(unique_genres[i], i) for i in range(unique_genres.shape[0])])
train_df['genre'] = train_df['genre'].map(genre_encoding)
train_df = train_df.sample(frac=1)
train_df

,title,genre,description,year,language,processed_description_string
48559,Partner to Partner,10,Jack Claves an alcoholic hard edged homicide ...,2011,English,jack clave alcohol hard edg homicid detect tea...
19346,Shvil Hahalav,0,"1964, a village in Galilee. The Mukhtar colla...",1997,English,1964 villag galile mukhtar collabor isra milit...
3676,Death of Film,3,Death of Film sings a eulogy for motion pictu...,2008,English,death film sing eulog motion pictur film comin...
22954,Rodnye,3,Russian citizen and Soviet-born Ukrainian nat...,2016,English,russian citizen soviet-born ukrainian nativ vi...
34562,Improv All Stars,4,"Drew Carey accompanied by Ryan Stiles, Colin ...",2001,English,drew carey accompani ryan stile colin mochri g...
...,...,...,...,...,...,...
46809,Le naufrage du Princess Sophia,3,This documentary explores the events surround...,2004,English,documentari explor event surround greatest mar...
39361,"""Family Feud""",23,"Family Feud is a family tv show for all ages,...",2014,English,famili feud famili tv show age whole famili en...
39021,Meng xing shi fen,0,"Ma Lei which sounds like ""Mary"" is a Chinese ...",1992,English,lei sound like mari chines citizen live hong k...
13958,Shanka,0,This story is about an honest police officer ...,1993,English,stori honest polic offic amit sen wife seema s...


In [7]:
num_labels = train_df['genre'].nunique()
num_labels

26

In [8]:
x = train_df['processed_description_string'].values
y = pd.get_dummies(train_df['genre']).values

In [9]:
train_set_size = 45000
val_set_size = 3000
test_set_size = 5000

x_train = x[:train_set_size]
x_val = x[train_set_size:train_set_size+val_set_size]
x_test = x[train_set_size+val_set_size:train_set_size+val_set_size+test_set_size]

y_train = y[:train_set_size]
y_val = y[train_set_size:train_set_size+val_set_size]
y_test = y[train_set_size+val_set_size:train_set_size+val_set_size+test_set_size]

## Tokenization

In [10]:
num_words = 2**9
max_len = 2**5

tokenizer = Tokenizer(num_words=num_words, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(x_train)

x_train = tokenizer.texts_to_sequences(x_train)
x_train = pad_sequences(x_train, maxlen=max_len)

x_val = tokenizer.texts_to_sequences(x_val)
x_val = pad_sequences(x_val, maxlen=max_len)

x_test = tokenizer.texts_to_sequences(x_test)
x_test = pad_sequences(x_test, maxlen=max_len)

## NN Training

In [11]:
# model = load_model('../models/nn_model')
model = Sequential()
model.add(Dense(128))
model.add(Dropout(0.1))
model.add(Dense(256))
model.add(Dropout(0.1))
model.add(Dense(256))
model.add(Dropout(0.1))
model.add(Dense(128))
model.add(Dropout(0.1))
model.add(Dense(num_labels, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

2022-11-30 14:27:33.254635: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-30 14:27:33.255261: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-30 14:27:33.255426: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-30 14:27:33.255529: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [12]:
my_callbacks  = [EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=5,
                              mode='auto')]

history = model.fit(x_train, y_train,
                    epochs=50, batch_size=32,
                    validation_data=(x_val, y_val),
                    callbacks=my_callbacks,
                    verbose=1
                   )

Epoch 1/50
1407/1407 [==============================] - 3s 2ms/step - loss: 26.8505 - accuracy: 0.1627 - val_loss: 3.6347 - val_accuracy: 0.2390
Epoch 2/50
1407/1407 [==============================] - 2s 2ms/step - loss: 3.7050 - accuracy: 0.1971 - val_loss: 2.4440 - val_accuracy: 0.2713
Epoch 3/50
 398/1407 [=======>......................] - ETA: 1s - loss: 2.7688 - accuracy: 0.2290

KeyboardInterrupt: 

## NN Evaluation

In [29]:
model.evaluate(x_test, y_test)

157/157 [==============================] - 0s 1ms/step - loss: 2.3386 - accuracy: 0.2648


[2.338596820831299, 0.2648000121116638]

In [30]:
model.save('../models/nn_model')

INFO:tensorflow:Assets written to: ../models/nn_model/assets


INFO:tensorflow:Assets written to: ../models/nn_model/assets


## GRU Training

In [31]:
EMBEDDING_DIM = 2**5

# model = load_model('../models/gru_model')
model = Sequential()
model.add(Embedding(num_words, EMBEDDING_DIM, input_length=x_train.shape[1]))
model.add(GRU(256, dropout=0.1))
model.add(Dense(num_labels, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [32]:
my_callbacks  = [EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=5,
                              mode='auto')]

history = model.fit(x_train, y_train,
                    epochs=20, batch_size=32,
                    validation_data=(x_val, y_val),
                    callbacks=my_callbacks,
                    verbose=1
                   )

Epoch 1/20
1407/1407 [==============================] - 5s 3ms/step - loss: 1.9942 - accuracy: 0.4173 - val_loss: 1.8444 - val_accuracy: 0.4663
Epoch 2/20
1407/1407 [==============================] - 4s 3ms/step - loss: 1.7505 - accuracy: 0.4852 - val_loss: 1.7316 - val_accuracy: 0.4960
Epoch 3/20
1407/1407 [==============================] - 4s 3ms/step - loss: 1.6726 - accuracy: 0.5008 - val_loss: 1.6832 - val_accuracy: 0.5020
Epoch 4/20
1407/1407 [==============================] - 4s 3ms/step - loss: 1.6210 - accuracy: 0.5092 - val_loss: 1.6442 - val_accuracy: 0.5110
Epoch 5/20
1407/1407 [==============================] - 4s 3ms/step - loss: 1.5802 - accuracy: 0.5177 - val_loss: 1.6438 - val_accuracy: 0.5057
Epoch 6/20
1407/1407 [==============================] - 4s 3ms/step - loss: 1.5437 - accuracy: 0.5258 - val_loss: 1.6093 - val_accuracy: 0.5113
Epoch 7/20
1407/1407 [==============================] - 4s 3ms/step - loss: 1.5101 - accuracy: 0.5329 - val_loss: 1.6089 - val_accuracy:

## GRU Evaluation

In [33]:
model.evaluate(x_test, y_test)

157/157 [==============================] - 0s 2ms/step - loss: 1.6494 - accuracy: 0.5190


[1.649399757385254, 0.5189999938011169]

In [34]:
model.save('../models/gru_model')

INFO:tensorflow:Assets written to: ../models/gru_model/assets


INFO:tensorflow:Assets written to: ../models/gru_model/assets


## LSTM Training

In [35]:
EMBEDDING_DIM = 2**5

# model = load_model('../models/lstm_model')
model = Sequential()
model.add(Embedding(num_words, EMBEDDING_DIM, input_length=x_train.shape[1]))
model.add(LSTM(64, dropout=0.1))
model.add(Dense(num_labels, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [36]:
my_callbacks  = [EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=3,
                              mode='auto')]

history = model.fit(x_train, y_train,
                    epochs=10, batch_size=32,
                    validation_data=(x_val, y_val),
                    callbacks=my_callbacks,
                    verbose=1
                   )

Epoch 1/10
1407/1407 [==============================] - 5s 3ms/step - loss: 2.0685 - accuracy: 0.4032 - val_loss: 1.8698 - val_accuracy: 0.4660
Epoch 2/10
1407/1407 [==============================] - 4s 3ms/step - loss: 1.8112 - accuracy: 0.4678 - val_loss: 1.8031 - val_accuracy: 0.4823
Epoch 3/10
1407/1407 [==============================] - 4s 3ms/step - loss: 1.7408 - accuracy: 0.4872 - val_loss: 1.7328 - val_accuracy: 0.4977
Epoch 4/10
1407/1407 [==============================] - 4s 3ms/step - loss: 1.6787 - accuracy: 0.5015 - val_loss: 1.7270 - val_accuracy: 0.4940
Epoch 5/10
1407/1407 [==============================] - 4s 3ms/step - loss: 1.6402 - accuracy: 0.5088 - val_loss: 1.7068 - val_accuracy: 0.5003
Epoch 6/10
1407/1407 [==============================] - 4s 3ms/step - loss: 1.6111 - accuracy: 0.5139 - val_loss: 1.6700 - val_accuracy: 0.5007
Epoch 7/10
1407/1407 [==============================] - 4s 3ms/step - loss: 1.5845 - accuracy: 0.5200 - val_loss: 1.6402 - val_accuracy:

## LSTM Evaluation

In [37]:
model.evaluate(x_test, y_test)

157/157 [==============================] - 0s 2ms/step - loss: 1.6114 - accuracy: 0.5172


[1.611369252204895, 0.5171999931335449]

In [38]:
model.save('../models/lstm_model')

INFO:tensorflow:Assets written to: ../models/lstm_model/assets


INFO:tensorflow:Assets written to: ../models/lstm_model/assets
